## Bayes Classifier

In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from scipy.io import loadmat
import numpy as np
from sklearn.metrics import accuracy_score
from bayes_classifier import *


## Read and load data

In [2]:
def GetAllData():
    data_all = {}
    # data.mat
    # 200 subjects
    # 3 faces per subject
    # size: 24 x 21
    print('-'*10,'Data')
    data = loadmat('data.mat')
    data = data['face']
    print(data.shape)
    # LAbel each 200 classes with their index and set as label
    labels_data = []
    for i in range(data.shape[2]):
        labels_data.append('lbl'+str(i+1))

    data = np.moveaxis(data, -1, 0)
    print(data.shape)
    # Separate 3 faces for the 200 subjects
    data = data.reshape(200,3,(24*21))
    print(data.shape)
    data_all['data'] = (data,labels_data)
    print('-'*10,'Data')

    print('-'*10,'Pose')
    # 68 subjects
    # 13 images per subject (13 different poses)
    # size: 48 x 40
    pose = loadmat('pose.mat')
    pose = pose['pose']
    labels_pose = []
    for i in range(pose.shape[3]):
        labels_pose.append('lbl'+str(i+1))
    print(pose.shape)
    pose = np.moveaxis(np.moveaxis(pose,-1,0),-1,1)
    print(pose.shape)
    data_all['pose'] = (pose,labels_pose)
    print('-'*10,'Pose')

    print('-'*10,'Illum')
    # 68 subjects
    # 21 images per subject (21 different illuminations)
    # size: 48x40
    illum = loadmat('illumination.mat')
    illum = illum['illum']
    labels_illum = []
    print(illum.shape)
    for i in range(illum.shape[2]):
        labels_illum.append('lbl'+str(i+1))
    illum = np.moveaxis(np.moveaxis(illum,-1,0),-1,1)
    print(illum.shape)
    data_all['illum'] = (illum,labels_illum)
    print('-'*10,'Illum')

    return data_all


data_all = GetAllData()


---------- Data
(24, 21, 600)
(600, 24, 21)
(200, 3, 504)
---------- Data
---------- Pose
(48, 40, 13, 68)
(68, 13, 48, 40)
---------- Pose
---------- Illum
(1920, 21, 68)
(68, 21, 1920)
---------- Illum


## Generate random train and test samples

In [23]:
x_train = {}
y_train = {}
#  [data,pose,illum]
n = [20,20,45]
m = [2,7,16]
#
c=0
train_test_all = {}  # Format-> Dataset: (TrainX, TrainY, TestX, TestY)
# Can test the effect of expressions, illumination variations. 
# Here we test the effect of illumination for data.mat
for k in data_all:
    train_datax = data_all[k][0][:,:m[c],:]
    train_datay = data_all[k][1]
    samples = range(data_all[k][0].shape[0])
    # ndarray generated random samples for test data
    rand_indexs = np.array(np.random.choice(samples, n[c], replace = False))
    test_datax = data_all[k][0][rand_indexs,m[c],:]
    test_datay = np.array(data_all[k][1])[rand_indexs]
    train_test_all[k] = (train_datax,train_datay,test_datax,test_datay)
    c+=1
    pass

#train_test_all

In [24]:
train_test_all['data'][0].shape

(200, 2, 504)

## Run without dimensionality reduction:

>> Data.mat

In [18]:
# Add noise = 0.1 to avoid singularity whne calculating covariance.
classifier = BayesClassifier(0.1)
predicted_data1,predicted_data2 = classifier.classify(train_test_all['data'][0],train_test_all['data'][2])
print(accuracy_score(predicted_data2,train_test_all['data'][3]))


0.6


>> Pose

In [19]:
# Add noise = 0.1 to avoid singularity whne calculating covariance.
classifier = BayesClassifier(0.1)
train_x = train_test_all['pose'][0]
test_x = train_test_all['pose'][2]
train_x = train_x.reshape(train_x.shape[0],train_x.shape[1],train_x.shape[2]*train_x.shape[3])
test_x = test_x.reshape(test_x.shape[0],test_x.shape[1]*test_x.shape[2])
predicted_pose1,predicted_pose2 = classifier.classify(train_x,test_x)
print(accuracy_score(predicted_pose2,train_test_all['pose'][3]))

0.8


>> Illumination

In [20]:
# Add noise = 0.1 to avoid singularity whne calculating covariance.
classifier = BayesClassifier(0.1)
predicted_illum1,predicted_illum2 = classifier.classify(train_test_all['illum'][0],train_test_all['illum'][2])
print(accuracy_score(predicted_illum2,train_test_all['illum'][3]))

1.0


## Principle Component Analysis

In [8]:
from pca import *

>> Data.mat

In [9]:
# Try with different number of components
num_comp = [10,30,70,200,250]
train_x = train_test_all['data'][0]
test_x = train_test_all['data'][2]
# Resize to 2D:
acc_bayes_pca_data = []
train_x = train_x.reshape(train_x.shape[0]*train_x.shape[1],train_x.shape[2])
for num in num_comp:
    pca = PCA(num)
    pca.fit(train_x)
    train_x_dash = pca.transform(train_x)
    # pca = PCA(num)
    # pca.fit(test_x)
    test_x_dash = pca.transform(test_x)
    # print(train_x_dash.shape)
    # print(test_x_dash.shape)
    train_x_dash = train_x_dash.reshape(200,2,num)

    classifier = BayesClassifier(0.75)
    predicted_data_pca1,predicted_data_pca2 = classifier.classify(train_x_dash,test_x_dash)
    acc = accuracy_score(predicted_data_pca2,train_test_all['data'][3])
    print('Component: ',num,' Accuraccy: ',acc)
    acc_bayes_pca_data.append(acc)
    pass


Component:  10  Accuraccy:  0.2
Component:  30  Accuraccy:  0.4666666666666667
Component:  70  Accuraccy:  0.6
Component:  200  Accuraccy:  0.7333333333333333
Component:  250  Accuraccy:  0.7333333333333333


>> Pose

In [25]:
# Try with different number of components
num_comp = [10,30,70,200,250]
train_x = train_test_all['pose'][0]
test_x = train_test_all['pose'][2]
# Resize to 2D:
train_x = train_x.reshape(train_x.shape[0]*train_x.shape[1],train_x.shape[2]*train_x.shape[3])
test_x = test_x.reshape(test_x.shape[0],test_x.shape[1]*test_x.shape[2])
acc_bayes_pca_pose = []
for num in num_comp:
    pca = PCA(num)
    pca.fit(train_x)
    train_x_dash = pca.transform(train_x)
    test_x_dash = pca.transform(test_x)
    # print(train_x_dash.shape)
    # print(test_x_dash.shape)
    train_x_dash = train_x_dash.reshape(68,7,num)

    classifier = BayesClassifier(0.1)
    predicted_pose_pca1,predicted_pose_pca2 = classifier.classify(train_x_dash,test_x_dash)
    acc = accuracy_score(predicted_pose_pca2,train_test_all['pose'][3])
    print(num,':\t',acc)
    acc_bayes_pca_pose.append(acc)
    pass

10 :	 0.7
30 :	 1.0
70 :	 1.0
200 :	 1.0
250 :	 1.0


>> Illum

In [26]:
# Try with different number of components
num_comp = [10,30,70,200,250]
train_x = train_test_all['illum'][0]
test_x = train_test_all['illum'][2]
# Resize to 2D:
train_x = train_x.reshape(train_x.shape[0]*train_x.shape[1],train_x.shape[2])
for num in num_comp:
    pca = PCA(num)
    pca.fit(train_x)
    train_x_dash = pca.transform(train_x)
    test_x_dash = pca.transform(test_x)
    # print(train_x_dash.shape)
    # print(test_x_dash.shape)
    train_x_dash = train_x_dash.reshape(68,16,num)

    classifier = BayesClassifier(0.15)
    predicted_illum_pca1,predicted_illum_pca2 = classifier.classify(train_x_dash,test_x_dash)
    print(accuracy_score(predicted_illum_pca2,train_test_all['illum'][3]))
    pass


0.4222222222222222
1.0
1.0
1.0
1.0


## Multiple Discriminant Analysis (MDA)

In [12]:
from mda import *

>> Data.mat

In [13]:
train_x = train_test_all['data'][0]
test_x = train_test_all['data'][2]
num_newdim = [10,30,70,200,250]
for n_dim in num_newdim:
    mda_data = MDA(0.9,train_x,test_x,n_dim)
    mda_data.get_matA()
    train_x_dash = mda_data.transform(train_x)
    test_x_dash = mda_data.transform(test_x)

    # Add noise = 0.1 to avoid singularity whne calculating covariance.
    classifier = BayesClassifier(0.95)
    predicted_data_mda1,predicted_data_mda2 = classifier.classify(train_x_dash,test_x_dash)
    print(accuracy_score(predicted_data_mda2,train_test_all['data'][3]))


0.2
0.4666666666666667
0.7333333333333333
0.7333333333333333
0.7333333333333333


>> Pose

In [14]:
train_x = train_test_all['pose'][0]
test_x = train_test_all['pose'][2]
train_x = train_x.reshape(train_x.shape[0],train_x.shape[1],train_x.shape[2]*train_x.shape[3])
test_x = test_x.reshape(test_x.shape[0],test_x.shape[1]*test_x.shape[2])
num_newdim = [10,30,70,200,250]
for n_dim in num_newdim:
    mda_pose = MDA(0.5,train_x,test_x,n_dim)
    mda_pose.get_matA()
    train_x_dash = mda_pose.transform(train_x)
    test_x_dash = mda_pose.transform(test_x)
    # Add noise = 0.8 to avoid singularity whne calculating covariance.
    classifier = BayesClassifier(0.8)
    predicted_pose_mda1,predicted_pose_mda2 = classifier.classify(train_x_dash,test_x_dash)
    print(accuracy_score(predicted_pose_mda2,train_test_all['pose'][3]))

0.7333333333333333
0.9333333333333333
0.9333333333333333
0.9333333333333333
0.9333333333333333


>> Illumination

In [27]:
train_x = train_test_all['illum'][0]
test_x = train_test_all['illum'][2]
acc_bayes_MDA_illum = []
num_newdim = [10,30,70,200,250]
for n_dim in num_newdim:
    mda_illum = MDA(0.1,train_x,test_x,n_dim)
    mda_illum.get_matA()
    train_x_dash = mda_illum.transform(train_x)
    test_x_dash = mda_illum.transform(test_x)

    classifier = BayesClassifier(0.75)
    predicted_illum_mda1,predicted_illum_mda2 = classifier.classify(train_x_dash,test_x_dash)
    acc = accuracy_score(predicted_illum_mda2,train_test_all['illum'][3])
    print(n_dim,'\t:',acc)
    acc_bayes_MDA_illum.append(acc)

10 	: 0.8
30 	: 1.0
70 	: 1.0
200 	: 1.0
250 	: 1.0
